In [1]:
def install_and_import_stanza():
    """Install and import Stanza library."""
    import os
    try:
        import stanza
    except ImportError:
        os.system('pip install stanza')
        import stanza
    return stanza


In [2]:
def download_tamil_model(stanza):
    """Download the Tamil language model for Stanza."""
    stanza.download('ta')  # Download if not already done

In [3]:
def initialize_tamil_pipeline(stanza, use_gpu=False):

    return stanza.Pipeline('ta', processors='tokenize,pos', use_gpu=use_gpu)

In [4]:
def process_tamil_text(pipeline, text):

    doc = pipeline(text)
    pos_tags = [(word.text, word.upos) for sentence in doc.sentences for word in sentence.words]
    return pos_tags


In [5]:
# Define the grammar correction function
def correct_grammar(pos_tags):

    words, tags = zip(*pos_tags)
    words = list(words)  # Convert tuple to list for modifications
    errors = []

    # Rule 1: Subject-Object-Verb (SOV) Order
    if "PRON" in tags and "NOUN" in tags and "VERB" in tags:
        pron_index = tags.index("PRON")
        noun_index = tags.index("NOUN")
        verb_index = tags.index("VERB")
        if not (pron_index < noun_index < verb_index):
            errors.append("Error: Subject-Object-Verb (SOV) order is incorrect.")
            # Reorder words to follow SOV
            reordered_words = [words[pron_index], words[noun_index], words[verb_index]]
            remaining_words = [w for i, w in enumerate(words) if i not in (pron_index, noun_index, verb_index)]
            words = reordered_words + remaining_words

    # Rule 2: Adjective-Noun Order
    if "ADJ" in tags and "NOUN" in tags:
        for i, tag in enumerate(tags):
            if tag == "ADJ":
                adj_index = i
                # Look for a noun after the adjective
                for j in range(adj_index + 1, len(tags)):
                    if tags[j] == "NOUN" and adj_index > j:
                        errors.append("Error: Adjective should precede the noun.")
                        # Swap adjective and noun
                        words[adj_index], words[j] = words[j], words[adj_index]

    # Rule 3: Plural Agreement
    if "PRON" in tags and "VERB" in tags:
        pron_index = tags.index("PRON")
        verb_index = tags.index("VERB")
        pron_word = words[pron_index]
        verb_word = words[verb_index]
        if pron_word.endswith("ள்") and not verb_word.endswith("ோம்"):
            errors.append("Error: Plural pronoun does not match plural verb.")
            # Correct the verb to plural form
            if "ேன்" in verb_word:
                words[verb_index] = verb_word.replace("ேன்", "ோம்")
            else:
                words[verb_index] += "ோம்"

    corrected_sentence = " ".join(words)
    return corrected_sentence, errors

# Extend process_tamil_text to include correction
def process_tamil_text_with_correction(pipeline, tamil_text):
    """
    Process Tamil text for POS tagging and correct grammatical errors.

    Parameters:
    - pipeline: Stanza NLP pipeline for Tamil.
    - tamil_text: str, the input Tamil text.

    Returns:
    - corrected_text: str, the corrected Tamil text.
    - errors: List of detected grammatical errors.
    """
    doc = pipeline(tamil_text)
    pos_tags = [(word.text, word.upos) for sent in doc.sentences for word in sent.words]

    corrected_text, errors = correct_grammar(pos_tags)
    return corrected_text, errors


In [6]:
if __name__ == "__main__":
    # Step 1: Install and Import Stanza
    stanza = install_and_import_stanza()

    # Step 2: Download Tamil Model
    download_tamil_model(stanza)

    # Step 3: Initialize Pipeline
    pipeline = initialize_tamil_pipeline(stanza, use_gpu=False)

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ta (Tamil) ...


INFO:stanza:Downloaded file to /root/stanza_resources/ta/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: ta (Tamil):
| Processor | Package      |
----------------------------
| tokenize  | ttb          |
| mwt       | ttb          |
| pos       | ttb_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they ar

In [7]:

    tamil_text = "நாங்கள் பாடம் படிக்கிறேன்"  # Example with grammatical errors


    corrected_text, errors = process_tamil_text_with_correction(pipeline, tamil_text)


    print("Original Text:", tamil_text)
    if errors:
        print("\nDetected Grammar Errors:")
        for error in errors:
            print(f"- {error}")
        print("\nCorrected Sentence:", corrected_text)
    else:
        print("\nThe sentence is grammatically correct!")

Original Text: நாங்கள் பாடம் படிக்கிறேன்

Detected Grammar Errors:
- Error: Plural pronoun does not match plural verb.

Corrected Sentence: நாங்கள் பாடம் படிக்கிறோம்


In [9]:

    tamil_text = "பாடம் நாங்கள் படிக்கிறோம்"  # Example with grammatical errors


    corrected_text, errors = process_tamil_text_with_correction(pipeline, tamil_text)


    print("Original Text:", tamil_text)
    if errors:
        print("\nDetected Grammar Errors:")
        for error in errors:
            print(f"- {error}")
        print("\nCorrected Sentence:", corrected_text)
    else:
        print("\nThe sentence is grammatically correct!")

Original Text: பாடம் நாங்கள் படிக்கிறோம்

Detected Grammar Errors:
- Error: Subject-Object-Verb (SOV) order is incorrect.

Corrected Sentence: நாங்கள் பாடம் படிக்கிறோம்


In [13]:

    tamil_text = "அவள் பாடம் நன்றாகப் படித்தார்"  # Example with grammatical errors


    corrected_text, errors = process_tamil_text_with_correction(pipeline, tamil_text)


    print("Original Text:", tamil_text)
    if errors:
        print("\nDetected Grammar Errors:")
        for error in errors:
            print(f"- {error}")
        print("\nCorrected Sentence:", corrected_text)
    else:
        print("\nThe sentence is grammatically correct!")

Original Text: அவள் பாடம் நன்றாகப் படித்தார்

The sentence is grammatically correct!
